In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pydicom
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from PIL import Image
from imblearn.over_sampling import SMOTE
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, RandomZoom
from imblearn.over_sampling import RandomOverSampler

import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns', 500)

/Users/johngalvin/miniforge3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Notes

This notebook has the same models as multimodal_embeddings.ipynb, except that instead of pooling 160 of each patient's images into a single vector, we perform feature extraction on the first 20 of each patient's images and feed **all** of them to the model.

## Generate embeddings for ADNI images

In [2]:
pt_ids = []
pixels = []

directory_path = '/Users/johngalvin/Downloads/ADNI 2'

# Iterate through level 2 subdirectories
for level_2_foldername in os.listdir(directory_path):
    level_2_folder_path = os.path.join(directory_path, level_2_foldername)
    
    if os.path.isdir(level_2_folder_path):
        # Iterate through DICOM files in level 5 (bottom-most level) of each level 2 folder
        for root, _, files in os.walk(level_2_folder_path):
            for file in files:
                try:
                    file_path = os.path.abspath(os.path.join(root, file))
                    
                    # Attempt to read DICOM file
                    dcm = pydicom.dcmread(file_path)
                    
                    # Check if the file has PixelData (to avoid non-image DICOM files)
                    if hasattr(dcm, 'PixelData'):
                        # Append both level 2 folder name and pixel array to the lists
                        pt_ids.append(file[5:15])
                        pixels.append(dcm.pixel_array)
                except Exception as e:
                    # Handle exceptions (e.g., files without 'TransferSyntaxUID')
                    print(f"Error processing file {file_path}: {e}")

# Create a DataFrame from the lists
mri_df = pd.DataFrame({'PTID': pt_ids, 'Pixel Array': pixels})

Error processing file /Users/johngalvin/Downloads/ADNI 2/068_S_0473/.DS_Store: File is missing DICOM File Meta Information header or the 'DICM' prefix is missing from the header. Use force=True to force reading.
Error processing file /Users/johngalvin/Downloads/ADNI 2/068_S_0473/MPRAGE/.DS_Store: File is missing DICOM File Meta Information header or the 'DICM' prefix is missing from the header. Use force=True to force reading.
Error processing file /Users/johngalvin/Downloads/ADNI 2/032_S_0677/.DS_Store: File is missing DICOM File Meta Information header or the 'DICM' prefix is missing from the header. Use force=True to force reading.
Error processing file /Users/johngalvin/Downloads/ADNI 2/032_S_0677/MPRAGE/.DS_Store: File is missing DICOM File Meta Information header or the 'DICM' prefix is missing from the header. Use force=True to force reading.
Error processing file /Users/johngalvin/Downloads/ADNI 2/032_S_0677/MPRAGE/2016-07-22_09_23_31.0/.DS_Store: File is missing DICOM File Met

In [3]:
# Resize image arrays with Bilinear Interpolation
resized_arrays = []

for val in mri_df["Pixel Array"]:
    image = Image.fromarray(val, mode='L')
    resized_image = image.resize((224, 224), Image.BILINEAR)
    resized_array = np.expand_dims(np.array(resized_image, dtype=np.uint8), axis=-1) #TF expects channel dim
    resized_arrays.append(resized_array)
    
mri_df["Pixel Array"] = resized_arrays

In [4]:
# Keep just the first 20 images for each patient (size/speed)
mri_df = mri_df.groupby("PTID").head(20)
mri_df.reset_index(inplace=True)

In [5]:
# Add 3 channels for VGG16 (num_samples, 224, 224, 3) - after running this cell
resized_arrays = []

for i in range(len(mri_df["Pixel Array"])):
    resized_arrays.append(np.repeat(mri_df["Pixel Array"][i], 3, axis=-1))
    
mri_df["Pixel Array"] = resized_arrays

In [6]:
# Read in med/famhist
mf_hist = pd.read_csv('../data/clinical_training_data_with_medhist_famhist.csv')

# Handle Nan
mf_hist["Family_History_of_AD"] = mf_hist["Family_History_of_AD"].fillna(0)
mf_hist["Family_History_of_Dementia"] = mf_hist["Family_History_of_Dementia"].fillna(0)

# For converting categorical variables to ints
label_encoder = LabelEncoder()
scaler = MinMaxScaler()

# Split features / target
X = mf_hist.drop(columns=['AD_dx_in_5_yrs'])
y = mf_hist['AD_dx_in_5_yrs']

# Encode features
X["Diagnosis_at_Baseline"] = label_encoder.fit_transform(X["Diagnosis_at_Baseline"])
X["Gender"] = label_encoder.fit_transform(X["Gender"])
X["Ethnicity"] = label_encoder.fit_transform(X["Ethnicity"])
X["Race"] = label_encoder.fit_transform(X["Race"])

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Scale pixel data
train_unscaled_mri_df = pd.merge(mri_df, X_train[["PTID"]], on="PTID", how="inner")
test_unscaled_mri_df = pd.merge(mri_df, X_test[["PTID"]], on="PTID", how="inner")

train_unscaled_arrays = np.array(train_unscaled_mri_df["Pixel Array"].tolist())
test_unscaled_arrays = np.array(test_unscaled_mri_df["Pixel Array"].tolist())

mean = np.mean(train_unscaled_arrays, axis=(0, 1, 2))
std = np.std(train_unscaled_arrays, axis=(0, 1, 2))

train_scaled_array = (train_unscaled_arrays - mean) / std
test_scaled_array = (test_unscaled_arrays - mean) / std

In [8]:
# Reorder labels (order changed when forming unscaled_mri_df)
y_train_final = []
for val in train_unscaled_mri_df["PTID"].value_counts().index.values:
    y_train_final.append(mf_hist.loc[mf_hist["PTID"] == val, "AD_dx_in_5_yrs"].values[0])
y_train_final = np.array(y_train_final)

y_test_final = []
for val in test_unscaled_mri_df["PTID"].value_counts().index.values:
    y_test_final.append(mf_hist.loc[mf_hist["PTID"] == val, "AD_dx_in_5_yrs"].values[0])
y_test_final = np.array(y_test_final)

In [9]:
print (f'Mean: {mean}')
print (f'STD: {std}')

Mean: [23.53734755 23.53734755 23.53734755]
STD: [31.01920724 31.01920724 31.01920724]


In [10]:
# Load model
feature_extractor = tf.keras.models.load_model("feature_extractor/feature_extractor_model")

2023-10-11 11:21:19.517396: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-11 11:21:19.517445: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M2 Pro


## Generate embeddings and pool

In [11]:
# Generate embeddings and pool
patient_ids = []
latent_reps = []

start = 0
end = 20
for i in range(len(train_unscaled_mri_df["PTID"].value_counts().index.values)):
    p_id = train_unscaled_mri_df["PTID"].value_counts().index.values[i]
    patient_ids.append(p_id)
    features = feature_extractor(train_scaled_array[start:end]).numpy() # Each patient has 20 images (20, 512)
    #latent_rep = np.mean(np.stack(features, axis=-1), axis=1) # (512,), max pooling
    latent_rep = features.flatten()
    latent_reps.append(latent_rep)
    start += 20
    end += 20

train_embeddings = pd.DataFrame()
train_embeddings["PTID"] = patient_ids
train_embeddings["embedding"] = latent_reps

In [12]:
# Generate embeddings and pool
patient_ids = []
latent_reps = []

start = 0
end = 20
for i in range(len(test_unscaled_mri_df["PTID"].value_counts().index.values)):
    p_id = test_unscaled_mri_df["PTID"].value_counts().index.values[i]
    patient_ids.append(p_id)
    features = feature_extractor(test_scaled_array[start:end]).numpy() # Each patient has 160 images (20, 512)
    #latent_rep = np.mean(np.stack(features, axis=-1), axis=1) # (512,), max pooling
    latent_rep = features.flatten()
    latent_reps.append(latent_rep)
    start += 20
    end += 20

test_embeddings = pd.DataFrame()
test_embeddings["PTID"] = patient_ids
test_embeddings["embedding"] = latent_reps

In [13]:
# Join back into X_train, X_test
X_train = pd.merge(train_embeddings, X_train, on="PTID")
X_test = pd.merge(test_embeddings, X_test, on="PTID")

# Scale remaining data (clinical/genetic)
embedding_column_train = X_train["embedding"]
embedding_column_test = X_test["embedding"]

columns_to_scale_train = X_train.drop(columns=["embedding", "PTID"])
columns_to_scale_test = X_test.drop(columns=["embedding", "PTID"])

scaled_train = scaler.fit_transform(columns_to_scale_train)
scaled_test = scaler.transform(columns_to_scale_test)

X_train_scaled = pd.DataFrame(data=scaled_train, columns=columns_to_scale_train.columns)
X_test_scaled = pd.DataFrame(data=scaled_test, columns=columns_to_scale_test.columns)

X_train_scaled["embedding"] = embedding_column_train
X_test_scaled["embedding"] = embedding_column_test

In [14]:
# Oversample
ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train_scaled, y_train_final)

## Concatenate embedding with clinical/genetic feature vector

In [15]:
# Structure for training
vector_column = X_train_resampled["embedding"].values
other_columns = X_train_resampled.drop(columns=["embedding"]).values
X_train_final = np.hstack((other_columns, np.vstack(vector_column)))

# Structure for test
vector_column = X_test_scaled["embedding"].values
other_columns = X_test_scaled.drop(columns=["embedding"]).values
X_test_final = np.hstack((other_columns, np.vstack(vector_column)))

In [16]:
IN_FEATURES = X_train_final[0].shape[0]

In [20]:
def create_model(dropout=0.3, learning_rate=0.0001, l2_penalty=0.1):
    """Builds classification model"""
    
    model = tf.keras.Sequential()
    inputs = tf.keras.layers.Input(shape=(IN_FEATURES,), name="input_layer") # (Batch, num_features)
    
    hidden_1 = tf.keras.layers.Dense(256, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(l2_penalty), name="hidden_1")(inputs)
    hidden_1 = tf.keras.layers.Dropout(dropout)(hidden_1)
    hidden_2 = tf.keras.layers.Dense(128, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(l2_penalty), name="hidden_2")(hidden_1)
    hidden_2 = tf.keras.layers.Dropout(dropout)(hidden_2)
    
    classification = tf.keras.layers.Dense(1, activation="sigmoid", name="classification_layer")(hidden_1)
    classification_model = tf.keras.Model(inputs=[inputs], outputs=[classification])
    
    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                 loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
                                 metrics="accuracy")

    return classification_model

In [21]:
model = create_model()

In [22]:
tf.config.run_functions_eagerly(True)
history = model.fit(X_train_final,
                    y_train_resampled,
                    #validation_data=[X_test_final, y_test_final],
                    batch_size=16,
                    epochs=100)

Epoch 1/100
22/22 [==============================] - 1s 23ms/step - loss: 45.0726 - accuracy: 0.5529
Epoch 2/100
22/22 [==============================] - 0s 22ms/step - loss: 34.4453 - accuracy: 0.6059
Epoch 3/100
22/22 [==============================] - 0s 22ms/step - loss: 26.1435 - accuracy: 0.6471
Epoch 4/100
22/22 [==============================] - 1s 23ms/step - loss: 19.8390 - accuracy: 0.6441
Epoch 5/100
22/22 [==============================] - 1s 24ms/step - loss: 14.9985 - accuracy: 0.6971
Epoch 6/100
22/22 [==============================] - 1s 23ms/step - loss: 11.3780 - accuracy: 0.7029
Epoch 7/100
22/22 [==============================] - 0s 22ms/step - loss: 8.6497 - accuracy: 0.7118
Epoch 8/100
22/22 [==============================] - 0s 21ms/step - loss: 6.6046 - accuracy: 0.7235
Epoch 9/100
22/22 [==============================] - 0s 22ms/step - loss: 5.0898 - accuracy: 0.7706
Epoch 10/100
22/22 [==============================] - 0s 22ms/step - loss: 3.9813 - accuracy: 

22/22 [==============================] - 0s 21ms/step - loss: 0.5037 - accuracy: 0.9000
Epoch 83/100
22/22 [==============================] - 0s 21ms/step - loss: 0.5056 - accuracy: 0.9147
Epoch 84/100
22/22 [==============================] - 0s 21ms/step - loss: 0.5033 - accuracy: 0.9059
Epoch 85/100
22/22 [==============================] - 0s 22ms/step - loss: 0.5092 - accuracy: 0.8824
Epoch 86/100
22/22 [==============================] - 0s 21ms/step - loss: 0.5342 - accuracy: 0.8382
Epoch 87/100
22/22 [==============================] - 0s 21ms/step - loss: 0.5506 - accuracy: 0.8412
Epoch 88/100
22/22 [==============================] - 0s 21ms/step - loss: 0.5347 - accuracy: 0.8529
Epoch 89/100
22/22 [==============================] - 0s 21ms/step - loss: 0.4993 - accuracy: 0.8941
Epoch 90/100
22/22 [==============================] - 0s 21ms/step - loss: 0.5032 - accuracy: 0.9059
Epoch 91/100
22/22 [==============================] - 0s 23ms/step - loss: 0.4884 - accuracy: 0.8971
Epo

In [23]:
model.evaluate(X_test_final, y_test_final)

2/2 [==============================] - 0s 43ms/step - loss: 0.7770 - accuracy: 0.6825


[0.7770423889160156, 0.682539701461792]

In [24]:
print ("If simply guessing all 0, can get this accuracy:", 1 - y_test_final.sum() / y_test_final.shape[0])

If simply guessing all 0, can get this accuracy: 0.746031746031746


# Try SVC

In [20]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

In [92]:
# Instantiate Model
svc = SVC()

In [93]:
# Grid search
param_grid = {
    "C": [0.1, 1, 10],                
    "kernel": ["linear", "poly", "rbf", "sigmoid"],
    "gamma": ["scale", "auto", 0.1],
    "degree": [1, 2, 3, 4, 5]}

In [94]:
grid_search = GridSearchCV(estimator=svc, param_grid=param_grid, cv=5, scoring="accuracy")

In [95]:
grid_search.fit(X_train_final, y_train_resampled)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10], 'degree': [1, 2, 3, 4, 5],
                         'gamma': ['scale', 'auto', 0.1],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             scoring='accuracy')

In [96]:
# Get the best parameters and best accuracy score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Print the best parameters and best score
print("Best Parameters:", best_params)
print("Best Accuracy:", best_score)

Best Parameters: {'C': 1, 'degree': 1, 'gamma': 0.1, 'kernel': 'rbf'}
Best Accuracy: 0.9147058823529411


In [97]:
# Instantiate and fit model
svc = SVC(C=1, degree=1, gamma=0.1, kernel="rbf")

svc.fit(X_train_final, y_train_resampled)

SVC(C=1, degree=1, gamma=0.1)

In [98]:
# Assess
y_pred = svc.predict(X_test_final)

# evaluate predictions
print(classification_report(y_test_final, y_pred))

              precision    recall  f1-score   support

           0       0.75      1.00      0.85        47
           1       0.00      0.00      0.00        16

    accuracy                           0.75        63
   macro avg       0.37      0.50      0.43        63
weighted avg       0.56      0.75      0.64        63



/Users/johngalvin/miniforge3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/johngalvin/miniforge3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/johngalvin/miniforge3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# What if we do image-only?

Note: this model below accepts the pooled embeddings formed after VGG16 feature extraction. The only difference from above is that it does **not** concatenate each final pooled embedding with the clinical/genetic feature vector.

In [29]:
X_train_img_only = np.vstack(X_train_scaled["embedding"])
y_train_img_only = y_train_final

X_test_img_only = np.vstack(X_test_scaled["embedding"])
y_test_img_only = y_test_final

In [36]:
def create_model(dropout=0.3, learning_rate=0.0001):
    """Builds classification model"""
    
    model = tf.keras.Sequential()
    inputs = tf.keras.layers.Input(shape=(10240,), name="input_layer") # (Batch, num_features)
    
    hidden_1 = tf.keras.layers.Dense(512, activation="relu", name="hidden_1")(inputs)
    hidden_1 = tf.keras.layers.Dropout(dropout)(hidden_1)
    hidden_2 = tf.keras.layers.Dense(256, activation="relu", name="hidden_2")(hidden_1)
    hidden_2 = tf.keras.layers.Dropout(dropout)(hidden_2)
    
    classification = tf.keras.layers.Dense(1, activation="sigmoid", name="classification_layer")(hidden_2)
    classification_model = tf.keras.Model(inputs=[inputs], outputs=[classification])
    
    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                 loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
                                 metrics="accuracy")

    return classification_model

In [37]:
img_only_model = create_model()

In [38]:
tf.config.run_functions_eagerly(True)
img_only_history = img_only_model.fit(X_train_img_only,
                                      y_train_img_only,
                                      validation_data=[X_test_img_only, y_test_img_only],
                                      batch_size=16,
                                      epochs=20)

Epoch 1/20
 3/16 [====>.........................] - ETA: 0s - loss: 0.7891 - accuracy: 0.5000

/Users/johngalvin/miniforge3/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
/Users/johngalvin/miniforge3/lib/python3.9/site-packages/keras/backend.py:5676: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


16/16 [==============================] - 1s 30ms/step - loss: 0.7068 - accuracy: 0.6194 - val_loss: 0.5726 - val_accuracy: 0.7460
Epoch 2/20
16/16 [==============================] - 0s 28ms/step - loss: 0.6386 - accuracy: 0.6640 - val_loss: 0.5837 - val_accuracy: 0.7460
Epoch 3/20
16/16 [==============================] - 0s 27ms/step - loss: 0.6211 - accuracy: 0.6883 - val_loss: 0.6174 - val_accuracy: 0.7302
Epoch 4/20
16/16 [==============================] - 0s 27ms/step - loss: 0.5979 - accuracy: 0.6721 - val_loss: 0.5738 - val_accuracy: 0.7460
Epoch 5/20
16/16 [==============================] - 0s 27ms/step - loss: 0.6007 - accuracy: 0.6761 - val_loss: 0.6881 - val_accuracy: 0.4921
Epoch 6/20
16/16 [==============================] - 0s 26ms/step - loss: 0.5940 - accuracy: 0.6802 - val_loss: 0.5886 - val_accuracy: 0.7460
Epoch 7/20
16/16 [==============================] - 0s 27ms/step - loss: 0.5553 - accuracy: 0.7287 - val_loss: 0.5814 - val_accuracy: 0.7460
Epoch 8/20
16/16 [======